In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_bokeh as pb
# from bokeh.plotting import figure, show
# from bokeh.io import output_notebook, export_png, output_file
# from bokeh.layouts import row, gridplot
# from bokeh.models import ColumnDataSource
import numpy as np
import random
import datetime
# import seaborn as sns; sns.set()
# %matplotlib inline
# pb.output_notebook()
# output_notebook()

In [2]:
starttime = datetime.datetime.now()

In [3]:
# 已知规则
# 1. 客户来源_原始 如果该字段为空，则自动填充为 展厅

In [2]:
filename = 'potential.202101.pkl'
monstr = filename.split('.')[1]

In [3]:
databookALL = pd.read_pickle("Data/%s" % filename, compression='bz2')

In [4]:
# databook = databookALL.drop(['车型', '大区', '客户来源更新', '月份'], axis=1)
databook = databookALL.copy()

In [5]:
databookALL['潜客量'].sum()

338134

In [6]:
# setting.xlsx 格式
# 每个sheet的名字会被用来作为字段映射时的字典名，所以最好用英文，比如现在setting里的sheet名就是"cartype","bigarea"
# 同时，sheet的内容是一个字段的映射关系，比如 大区名称->大区
#      A    B
#  1  东北区 A1
# 这样读取的时候就可以按照下面去读了
tExcel = pd.read_excel("Data/settingALL.xlsx", sheet_name=None)

In [7]:
# 这里settingHash就是未来做dataframe.map()用的
settingHash = {}
for keys in tExcel:
    settingHash[keys] = dict(tExcel[keys].values)

In [8]:
tExcel.keys()

dict_keys(['city', 'cname', 'bigarea', 'cartype'])

In [9]:
# 客户来源_原始 中 所有NaN值 都会赋值为展厅
databook['客户来源更新'] = databook['客户来源_原始'].fillna('展厅')

databook['车型new'] = databook['分组名称'].map(settingHash['cartype'])

In [12]:
# (2021-02-08) 保存原始车型信息，所以不用考虑空值
# map的方式举例，m没用，但是建议将其保留，备以后查
# 当有新的车型进来时，可以在这里看到。
# m = pd.DataFrame({'a':databook['分组名称'].head(200), 'b':databook['分组名称'].head(200)})
# m['b'] = m['a'].map(settingHash['cartype'])
# m.drop_duplicates(['a','b'],  'first')

In [10]:
def transArr(df):
    # 将 [[A4L, 10], [A6, 5]], 这种类型的改成10个A4L，5个A6
    # [A4L, A4L, A4L, A4L, A4L, A4L, A4L, A4L, A4L, A4L, A6, A6, A6, A6, A6]
    # 方便后面按照随机取数出来，这样取出来应该就是这种分布
    tmp = []
    for idx, row in df.iterrows():
        tmp.extend([row['车型new']] * row['潜客量'])
    return tmp

In [11]:
# 清空dataMiss时，要用这种办法
# dataMiss.drop(dataMiss.index, inplace=True)

In [12]:
def genNewData(dCode, cSource, nowPOS):
    # dCode 经销商代码
    # cSource 客户来源更新
    # 进来后再做循环
    # 这里需要注意，dataMiss到底有哪些列
    tmpArr = transArr(dataCOMPUTed[(dataCOMPUTed['经销商代码']==dCode)&(dataCOMPUTed['客户来源更新']==cSource)&(dataCOMPUTed['潜客量']>0)][['车型new', '潜客量']])
    for idx, row in dataBYday[(dataBYday['经销商代码']==dCode)&(dataBYday['客户来源更新']==cSource)].iterrows():
        toDel = []
        for aIDX in random.sample(range(len(tmpArr)), int(row['潜客量'])):
            dataMiss.loc[nowPOS] = [
                pd.to_datetime(row['日期']),
                row['大区名称'],
                row['小区名称'],
                row['经销商代码'],
                1,
                row['客户来源更新'],
                tmpArr[aIDX]
            ]
            nowPOS +=1 
            toDel.append(aIDX)
        for i in sorted(toDel, reverse=True):
            del(tmpArr[i])
    return nowPOS

In [13]:
def tempDF(lvl, df):
    tmp = df[[lvl, '客户来源更新', '车型new', '潜客量']].copy()
    tmp = tmp.groupby([lvl, '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
    tmp['%s汇总' % lvl] = tmp.groupby([lvl, '客户来源更新'])['潜客量'].transform('sum')
    tmp['车型占比'] = tmp['潜客量']/tmp['%s汇总' % lvl]
    tmp.drop(['潜客量', '%s汇总' % lvl], axis=1, inplace=True)
    return tmp

In [14]:
def batDraw(lvl):
    graphics = []
    vals = list(getIDX(lvl, dataUSEstd))
    df_o = tempDF(lvl, dataUSEstd)
    df_c = tempDF(lvl, dataEsti)

    for t in ["展厅", "线上"]:
        tmp = []
        for i in vals:
            tmp.append(drawGraph(
                df_o[(df_o[lvl]==i)&(df_o['客户来源更新']==t)][['车型new', '车型占比']],
                df_c[(df_c[lvl]==i)&(df_c['客户来源更新']==t)][['车型new', '车型占比']],
                '%s %s' % (i, t)
            ))
        graphics.append(tmp)

    return graphics

In [15]:
def drawGraph(se1, se2, title='aa'):
    tmpdf = se1.merge(se2, on=['车型new'], suffixes=['_origin', '_fix'])
    tmpdf.set_index(['车型new'], inplace=True)
    m = tmpdf.plot_bokeh(kind='barh', legend='top_right', title=title, show_figure=False, zooming=False)
    return m

In [16]:
def resetType(ser):
    if pd.isnull(ser['分组名称']):
        return np.nan
    else:
        return ser['车型']

In [54]:
def covTypes(df):
    for col in df.columns:
        if col == '潜客量':
            df[col] = df[col].astype('int32')
        elif col== '日期':
            continue
        else:
            df[col] = df[col].astype('category')

In [18]:
def getIDX(cName, df, c=6):
    # 在df[cName]的最高30%里，随机取c个作为输出
    # 用于将来的分布验证
    tmp = df[[cName, '潜客量']].groupby([cName])['潜客量'].sum().reset_index()
    tmplist = df[cName].unique()
    if int(len(tmplist)*0.3) <= c:
        tmpidx = tmp.sort_values(by='潜客量', ascending=False).index.values
    else:
        tmpidx = tmp.sort_values(by='潜客量', ascending=False)[:int(len(tmplist)*0.3)].index.values
    return np.array(tmp[cName].iloc[random.sample(list(tmpidx), c)])

In [19]:
def balanceCom(rows):
    # 根据差值选择计算方式
    if rows['差值'] < 0:
        if rows['已进位'] and rows['顺序']+rows['差值']<=0:
            return rows['预计算潜客2'] - 1
    elif rows['差值'] > 0:
        if (not rows['已进位']) and rows['顺序'] - rows['差值'] <= 0:
            return rows['预计算潜客2'] + 1
    return rows['预计算潜客2']

In [20]:
def getColumns(col, vals,ty):
    # 获取画图的列
    oriData = dataUSEstd[(dataUSEstd[col]==vals)&(dataUSEstd['客户来源更新']==ty)][['车型new', '车型占比']]
    comData = dataEsti[(dataEsti[col]==vals)&(dataEsti['客户来源更新']==ty)][['车型new', '车型占比']]
    # oriData = statRES(col, dataUSEstd)[]
    return oriData.merge(comData, on='车型new', suffixes=['_origin', '_fix'])

In [55]:
covTypes(databook)

In [10]:
# 这里确实要删掉 分组名称，因为分组和车型new是多对一，如果不删，将来merge回来时，分组名称会为NaN
dataAvailable = databook[databook['车型new'].notna()]
dataAvailable = dataAvailable.drop(['分组名称', '客户来源_原始'], axis=1)

In [11]:
dataAvailable['潜客量'].sum(), databook[databook['车型new'].isna()]['潜客量'].sum()

(303777, 34357)

In [12]:
dataMiss = pd.DataFrame(columns=list(dataAvailable.columns))

In [13]:
# dataUSE = databook[['客户来源更新', '大区名称', '小区名称', '城市', '经销商代码', '车型new', '潜客量']].copy()
dataUSE = databook[['大区名称', '小区名称', '经销商代码', '潜客量', '客户来源更新', '车型new']]

In [14]:
# cityLIST = dataUSE['城市'].unique()
area1LIST = dataUSE['大区名称'].unique()
area2LIST = dataUSE['小区名称'].unique()
dealerLIST = dataUSE['经销商代码'].unique()
print(len(area1LIST), len(area2LIST), len(dealerLIST))#, len(cityLIST)

6 56 578


In [15]:
monthDays = len(databook['日期'].unique())

In [64]:
dataAvailable.groupby(by=['大区名称', '客户来源更新', '车型new']).mean()

潜客量
大区名称 客户来源更新 车型new            
东北区  展厅     A3 Lim   1.639752
            A3 SpB   1.636076
            A4 AR    1.279793
            A4 AV    1.131313
            A4L      3.148851
...                       ...
西部区  线上     Q5L Spb  1.051282
            Q7       1.129555
            Q8       1.069767
            S+Niche  1.555957
            e-tron   1.000000

[288 rows x 1 columns]

In [24]:
dataAvailable.head()

,日期,大区名称,小区名称,经销商代码,潜客量,客户来源更新,车型new
0,2021-01-01,北部区,北京1区,SA11003,1,线上,A3 SpB
1,2021-01-01,北部区,北京1区,SA11003,1,展厅,A4 AV
2,2021-01-01,北部区,北京1区,SA11003,1,展厅,A4L
3,2021-01-01,北部区,北京1区,SA11003,1,展厅,A5 SpB
4,2021-01-01,北部区,北京1区,SA11003,3,展厅,A6L


In [22]:
ooo = databook[databook['车型new'].isna()].copy()
ooo.info()
# ooo.groupby(['日期', '大区名称', '小区名称', '经销商代码'])['潜客量'].sum().head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8084 entries, 1384 to 154255
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   日期       8084 non-null   datetime64[ns]
 1   大区名称     8084 non-null   category      
 2   小区名称     8084 non-null   category      
 3   经销商代码    8084 non-null   category      
 4   分组名称     0 non-null      category      
 5   客户来源_原始  8084 non-null   category      
 6   潜客量      8084 non-null   int32         
 7   客户来源更新   8084 non-null   category      
 8   车型new    0 non-null      category      
dtypes: category(7), datetime64[ns](1), int32(1)
memory usage: 246.1 KB


In [51]:
dataBYday = databook[databook['车型new'].isna()].groupby(['日期', '大区名称', '经销商代码', '客户来源更新'])['潜客量'].sum().reset_index()

In [31]:
# 计算 经销商 客户来源 和车型 的group by sum，去掉日期来计算概率
# sum(经销商、客户来源、车型)/monthDays 这就是某经销商在某来源下车型出现的概率
# dealerDISTbd.groupby(["经销商代码", "客户来源更新", "车型new", "日期"])["潜客量"].sum().reset_index()
# 按日的做起来太麻烦所以不用这个了

In [32]:
dataUSEstd = dataUSE[dataUSE['车型new'].notna()]
dataUSE2b = dataUSE[dataUSE['车型new'].isna()]

In [33]:
# 汇总车型数据
dataUSEstd = dataUSEstd.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

In [34]:
# (2021-02-08)创建一个基于车型的城市分布，这里改成 大区分布 
# 因为后续会有部分经销商没有对应 "客户来源更新" 的分布，比如某些经销商没有线上的已有数据作为基准，这时就用城市的替换
# dataCITYstd = dataUSEstd.groupby(['城市', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
dataCITYstd = dataUSEstd.groupby(['大区名称', '车型new'])['潜客量'].sum().reset_index()

In [35]:
# 可能存在来源更新没有的情况，处理起来太复杂，直接用城市了
# dataCITYstd['城市汇总'] = dataCITYstd.groupby(['城市', '客户来源更新'])['潜客量'].transform('sum')
dataCITYstd['大区汇总'] = dataCITYstd.groupby(['大区名称'])['潜客量'].transform('sum')
dataCITYstd['车型占比'] = dataCITYstd['潜客量']/ dataCITYstd['大区汇总']

In [36]:
dataUSEstd['dealer汇总'] = dataUSEstd.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].transform('sum')
dataUSEstd['车型占比'] = dataUSEstd['潜客量']/dataUSEstd['dealer汇总']
dataUSEstd['日均概率'] = dataUSEstd['潜客量']/monthDays

In [37]:
dataUSE2b = dataUSE2b.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].sum().reset_index()

In [38]:
datatmp = dataUSE2b.copy()

In [ ]:
datatmp = datatmp.merge(dataUSEstd[["经销商代码", "客户来源更新", "车型new", "车型占比"]], on=["经销商代码", "客户来源更新"], how='left')

In [43]:
datatmp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388416 entries, 0 to 388415
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   大区名称    388416 non-null  category
 1   小区名称    388416 non-null  category
 2   经销商代码   388416 non-null  category
 3   客户来源更新  388416 non-null  category
 4   潜客量     388416 non-null  int32   
dtypes: category(4), int32(1)
memory usage: 3.4 MB


In [42]:
datatmp.merge(dataUSEstd[["经销商代码", "客户来源更新", "车型new", "车型占比"]], on=["经销商代码", "客户来源更新"], how='left')

MemoryError: Unable to allocate 23.3 GiB for an array with shape (3132186624,) and data type int64

In [ ]:
noneSTD = datatmp[datatmp['车型占比'].isna()]

In [ ]:
datatmp = datatmp[datatmp['车型占比'].notna()]

In [ ]:
noneSTD.drop(['车型new', '车型占比'], axis=1, inplace=True)

In [ ]:
noneSTD = noneSTD.merge(dataCITYstd[['大区名称', '车型new', '车型占比']], on=['大区名称'])

In [ ]:
datatmp = pd.concat([datatmp, noneSTD], ignore_index=True)

In [ ]:
# 预计算潜客1 是未四舍五入的值， 预计算潜客2 是四舍五入后的值
datatmp['预计算潜客'] = datatmp["潜客量"] * datatmp["车型占比"]
datatmp['预计算潜客1'] = datatmp['预计算潜客'].map(int)
datatmp['预计算潜客2'] = datatmp['预计算潜客'].apply(lambda x: int(x+0.5))
datatmp['进位可能'] = datatmp['预计算潜客'] - datatmp['预计算潜客1']

In [ ]:
# 这里计算出来的 差值 就是四舍五入后的差异，需要用这个差异去修改 datatmp
# 里的 预计算潜客2 中不合适的部分。
# 并在 datatmp 中插入新的列 调整潜客
# 所以需要利用 进位可能，进行多退少补：
#     如果需要退n个（差值为负），则寻找 进位可能 > 0.5的数据里最小的n个各去掉 1
#     如果需要补n个（差值为正），则寻找 进位可能 < 0.5的数据里最大的n个各增加 1
# 这部分需要使用函数计算，感觉还比较麻烦
# 这里直接做一个 进位可能 相关的函数，主要是插入 顺序 字段，值是序数：
#   大于 0.5 的 升序，比如：
#     进位可能: [0.51, 0.66, 0.501, 0.7]
#        顺序: [1, 2, 0, 3]
#   小于 0.5 的 降序
##########################################

In [ ]:
# 1. 给 datatemp 增加一列，区分 0.5
datatmp['已进位'] = datatmp['进位可能'] >= 0.5

In [ ]:
# 都使用的升序，处理时要注意
# 先处理 已进位 为 True的
datatmp['顺序'] = 0
l = datatmp['顺序'] + datatmp[datatmp['已进位']==True].groupby(['经销商代码', '客户来源更新', '已进位'])['进位可能'].rank(ascending=True, method="first").astype('int')
r = datatmp['顺序'] + datatmp[datatmp['已进位']==False].groupby(['经销商代码', '客户来源更新', '已进位'])['进位可能'].rank(ascending=False, method="first").astype('int')
datatmp['顺序'] = l.fillna(0) + r.fillna(0)

In [ ]:
# 计算经销商分来源的预计算潜客2的汇总
dataUSEpre = datatmp.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['预计算潜客2'].sum().reset_index()

In [ ]:
#dataUSEadj = datatmp.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['预计算潜客2'].sum().reset_index()
#dataUSEadj = dataUSE2b.head(200).merge(dataUSEpre[['经销商代码', '客户来源更新', '预计算潜客2']], on=['经销商代码', '客户来源更新'])
dataUSEadj = dataUSE2b.merge(dataUSEpre[['经销商代码', '客户来源更新', '预计算潜客2']], on=['经销商代码', '客户来源更新'])
dataUSEadj['差值'] = dataUSEadj['潜客量'] - dataUSEadj['预计算潜客2']

In [ ]:
# 将 差值 合并进 datatmp
datatmp = datatmp.merge(dataUSEadj[['经销商代码', '客户来源更新', '差值']], on=['经销商代码', '客户来源更新'])

In [ ]:
datatmp['预计算真值'] = datatmp.apply(balanceCom, axis=1)

In [ ]:
# 这里开始合并模拟结果和真实结果

In [ ]:
dataCOMPUTed = datatmp[['大区名称', '小区名称', '经销商代码', '客户来源更新','车型new', '预计算真值']].copy()

In [ ]:
dataCOMPUTed.rename(columns={'预计算真值': '潜客量'}, inplace=True)
dataCOMPUTed.head()

In [ ]:
dataEsti = dataCOMPUTed.append(dataUSEstd[['大区名称', '小区名称', '经销商代码', '客户来源更新','车型new', '潜客量']])

In [ ]:
dataEsti = dataEsti.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新','车型new'])['潜客量'].sum().reset_index()

In [ ]:
# 添加车型站比
dataEsti['dealer汇总'] = dataEsti.groupby(['大区名称', '小区名称', '经销商代码', '客户来源更新'])['潜客量'].transform('sum')
dataEsti['车型占比'] = dataEsti['潜客量']/dataEsti['dealer汇总']
dataEsti['日均概率'] = dataEsti['潜客量']/monthDays

In [ ]:
#### 以下是之前处理的结果

In [ ]:
def summarySTAT():
    totalPotentials = databook['潜客量'].sum()
    # dealPotentialsbyDay = databook.groupby(['经销商代码', '日期'])['潜客量'].sum().reset_index()
    totalDealPotentials = databook.groupby(['经销商代码'])['潜客量'].sum().reset_index()
    totalPotentialsFIX = dataEsti['潜客量'].sum()
    # dealPotentialsbyDayFIX = dataEsti.groupby(['经销商代码', '日期'])['潜客量'].sum().reset_index()
    totalDealPotentialsFIX = dataEsti.groupby(['经销商代码'])['潜客量'].sum().reset_index()
    # dealDiffbyDay = dealPotentialsbyDay.merge(dealPotentialsbyDayFIX, on=['经销商代码', '日期'], suffixes=['_origin', '_fix'])
    dealDiff = totalDealPotentials.merge(totalDealPotentialsFIX, on=['经销商代码'], suffixes=['_origin', '_fix'])
    # dealDiffbyDay['差值'] = dealDiffbyDay['潜客量_origin'] - dealDiffbyDay['潜客量_fix']
    dealDiff['差值'] = dealDiff['潜客量_origin'] - dealDiff['潜客量_fix']
    print(dealDiff['差值'].value_counts())
    return totalPotentials - totalPotentialsFIX, dealDiff['差值'].value_counts()

In [ ]:
dealerUnique = dataCOMPUTed[['大区名称', '小区名称', '经销商代码', '客户来源更新']].drop_duplicates()
dealerUnique = dealerUnique.reset_index(drop=True)[['经销商代码', '客户来源更新']]

In [ ]:
start = 0
for idx, row in dealerUnique.iterrows():
    print(idx, row['经销商代码'], row['客户来源更新'])
    start = genNewData(row['经销商代码'], row['客户来源更新'], start)
    print(start)

In [ ]:
dataMiss.head(1)

In [ ]:
# a1 = dataMiss.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

# a2 = a1.merge(dataCOMPUTed, on=['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新', '车型new'], suffixes=['_o', '_f'])

In [ ]:
# 图表所需的层次
# (2021-02-08)去掉了城市维度
# graphLVL = ["大区名称", "小区名称", "城市", "经销商代码"]
graphLVL = ["大区名称", "小区名称", "经销商代码"]

In [ ]:
pics = []
for pic in graphLVL:
    pics.extend(batDraw(pic))
html = pb.plot_grid(pics, plot_width=300, return_html=True)

In [ ]:
with open("tt.html", 'w') as f:
    f.write(html)

In [ ]:
# 轮盘赌算法，应该用不着了，直接用估计值去填坑
# import random
# maxnum = 10000
# pDIS = [0.046414, 0.050633, 0.147679, 0.147679, 0.270042, 0.008439, 0.004219, 0.033755, 0.025316, 0.07173, 0.004219, 0.008439, 0.181435]
# pDISsum = []
# for i in range(len(pDIS)):
#     if pDISsum:
#         pDISsum.append(pDISsum[-1] + pDIS[i])
#     else:
#         pDISsum.append(pDIS[i])
# result = [0] * len(pDIS)
# for i in range(maxnum):
#     rNUM = random.random()
#     for n, m in enumerate(pDISsum):
#         if rNUM < m:
#             result[n] += 1
#             break
# print(list(map(lambda x: '%.3f' % x, pDIS)))      
# print(list(map(lambda x: '%.3f' % (x/maxnum), result)))

In [ ]:
dataMiss['潜客量'].sum(), dataAvailable['潜客量'].sum()

In [ ]:
databookNew = dataAvailable.append(dataMiss, ignore_index=True)

In [ ]:
databookNew.head(1)

In [ ]:
# 这是一个总数检查，databookNew = databookALL
databookNew['潜客量'].sum(),databookALL['潜客量'].sum(), dataAvailable['潜客量'].sum(), dataMiss['潜客量'].sum()

In [ ]:
# databookNew = databookNew.groupby(['月份', '日期', '大区名称', '大区', '小区名称', '经销商代码', '经销商名称', '城市', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
databookNew = databookNew.groupby(['日期', '大区名称', '小区名称', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

In [ ]:
# 这里做 城市、经销商名称、大区 的生成
# databookNew['大区'] = databook['大区名称'].map(settingHash['bigarea'])
databookNew['大区'] = databookNew['大区名称'].map(settingHash['bigarea'])
databookNew['月份'] = databookNew['日期'].map(lambda x: str(x.month)+"月")
databookNew['经销商名称'] = databookNew['经销商代码'].map(settingHash['cname'])
databookNew['城市'] = databookNew['经销商代码'].map(settingHash['city'])

In [ ]:
# databookNew 做类型转换，减少体积
covTypes(databookNew)

In [ ]:
# 存两个文件
# 1. pkl压缩版，万一以后要操作就不必从excel中读取了
# 2. excel文件，分发给相关工作人员
databookNew.to_pickle('Results/potential.mani.%s.pkl' % monstr, compression='bz2')
databookNew.to_excel('Results/potential.%s.xlsx' % monstr, encoding='utf8', header=True, index=False)

In [ ]:
endtime = datetime.datetime.now()

In [ ]:
(endtime - starttime).seconds